<a href="https://colab.research.google.com/github/vjhawar12/Vision-Transformer-paper-implementation/blob/main/notebook/Vision_Transformer_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale

In [37]:
!git clone https://github.com/vjhawar12/Vision-Transformer-paper-implementation.git
%cd Vision-Transformer-paper-implementation

Cloning into 'Vision-Transformer-paper-implementation'...
remote: Enumerating objects: 174, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 174 (delta 93), reused 54 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (174/174), 367.95 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/Vision-Transformer-paper-implementation/Vision-Transformer-paper-implementation/Vision-Transformer-paper-implementation/Vision-Transformer-paper-implementation


In [38]:
import json
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from vit.dataloader import get_dataloaders
from vit.scheduler_func import sched_func
from vit.transforms import get_transforms
from vit.train_utils import train, validate
from vit.test_utils import test
from torchvision.transforms.v2 import CutMix, MixUp, RandomChoice
from vit.vit_model import PatchEmbedding, ManualMultiHeadSelfAttention, TransformerEncoder, VisionTransformer
from vit.data import data
from vit.dataset import get_training_data, get_test_data
from timm.loss import SoftTargetCrossEntropy
from torch.optim.lr_scheduler import LambdaLR
from pprint import pprint

In [49]:
torch.__version__

'2.6.0+cu124'

# Variable initialization

In [39]:
with open("hyperparams.json", "r") as f:
    params = json.load(f)
    pprint(params)

{'alpha': 0.25,
 'batchsize': 512,
 'channels': 3,
 'cutmix_or_mixup_start': 20,
 'dropout': 0.05,
 'embeddim': 252,
 'encoders': 8,
 'epochs': 100,
 'imsize': 64,
 'max_cutmix_or_mixup': 0.5,
 'max_mixup': 0.5,
 'mixup_start': 10,
 'numclasses': 10,
 'numheads': 12,
 'patchsize': 4}


In [40]:
train_transform, test_transform = get_transforms()

training_data = get_training_data(train_transform)
test_data = get_test_data(test_transform)

100%|██████████| 170M/170M [00:02<00:00, 81.3MB/s]


In [41]:
train_dataloader, test_dataloader = get_dataloaders(training_data, test_data, params['batchsize'])

In [42]:
loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
sft_loss_fn = SoftTargetCrossEntropy()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scaler = torch.amp.GradScaler()

cutmix = CutMix(num_classes=params['numclasses'])
mixup = MixUp(num_classes=params['numclasses'], alpha=params['alpha'])

# CUDA optimizations

In [43]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.enable_flash_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_math_sdp(True)

# Model creation

In [44]:
vit = torch.compile(VisionTransformer(
    batches=params['batchsize'], in_channels=params['channels'],
    patch_size=params['patchsize'], embed_dim=params['embeddim'],
    heads=params['numheads'], depth=params['encoders'], size=params['imsize'],
    num_classes=params['numclasses']
  ))

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit.to(device)

OptimizedModule(
  (_orig_mod): VisionTransformer(
    (patch_embedding): PatchEmbedding(
      (proj): Conv2d(3, 252, kernel_size=(4, 4), stride=(4, 4))
    )
    (transformer_stack): ModuleList(
      (0-7): 8 x TransformerEncoder(
        (mhsa): ManualMultiHeadSelfAttention(
          (Q_proj): Linear(in_features=252, out_features=252, bias=True)
          (V_proj): Linear(in_features=252, out_features=252, bias=True)
          (K_proj): Linear(in_features=252, out_features=252, bias=True)
          (dropout): Dropout(p=0.15, inplace=False)
          (output): Linear(in_features=252, out_features=252, bias=True)
        )
        (ln1): LayerNorm((252,), eps=1e-05, elementwise_affine=True)
        (drop_path_1): DropPath(drop_prob=0.050)
        (drop_path_2): DropPath(drop_prob=0.050)
        (ffn): Sequential(
          (0): Linear(in_features=252, out_features=252, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.15, inplace=False)
          (3): Lin

In [46]:
optim = torch.optim.AdamW(vit.parameters(), lr=1e-3, weight_decay=0.05)
scheduler = LambdaLR(optim, lr_lambda=sched_func)

# Train and validation set loop

In [47]:
train_acc = []
val_acc = []
lr = []
losses = []

for i in range(1):
#for i in range(params['epochs']):
    torch.cuda.empty_cache()
    vit.train(True)

    loop = tqdm(train_dataloader, desc=f"Epoch {i+1}/{params['epochs']}", leave=True, disable=False)

    total, correct, loss = train(
        vit, device, params['cutmix_or_mixup_start'],
        params['mixup_start'], params['max_cutmix_or_mixup'],
        params['max_mixup'], optim, mixup, cutmix,
        sft_loss_fn, loss_fn, scaler, i, loop,
    )

    val_total, val_correct = validate(vit, test_dataloader, device)

    val_accuracy = val_correct / val_total
    accuracy = correct / total

    scheduler.step()
    current_lr = optim.param_groups[0]['lr']

    losses.append(loss)
    lr.append(current_lr)
    train_acc.append(accuracy)
    val_acc.append(val_accuracy)

    print(f"Epoch {i + 1}: LR={current_lr:.6f} \t Train Acc: {accuracy:.4f} \t Val Acc: {val_accuracy:.4f}")

Epoch 1/100: 100%|██████████| 98/98 [00:20<00:00,  4.81it/s, loss=2.42]


Epoch 1: LR=0.000100 	 Train Acc: 10.3670 	 Val Acc: 11.6959


# Test set loop

In [48]:
vit.eval()
correct, total = test(vit, test_dataloader, device)
accuracy = correct / total

print(f"Accuracy: {accuracy}")

TypeError: autocast.__init__() missing 1 required positional argument: 'device_type'

# Per-epoch loss graph

In [ ]:
plt.title("Loss vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.plot(losses)

# Per-epoch LR graph

In [ ]:
plt.title("Learning Rate vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")

plt.plot(lr)

# Per-epoch Training vs Validation set accuracy graph

In [ ]:
plt.plot(train_acc, label="Train")
plt.plot(val_acc, label="Val")

plt.title("Accuracy vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")


plt.legend()

# Data table

In [ ]:
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', None)
df.transpose()

## 📊 Final Results

- **Dataset**: CIFAR-10
- **Model**: Vision Transformer (ViT)
- **Paper**: [An Image is Worth 16x16 Words (Dosovitskiy et al.)](https://arxiv.org/abs/2010.11929)
- **Training Epochs**: 100
- **Batch Size**: 512  
- **Optimizer**: AdamW  
- **Learning Rate**: 0.003 with 0.05 weight decay
- **Train Accuracy**: 87.2%
- **Test Accuracy**: 85.7%

